In [7]:
import pandas as pd
import numpy as np
import re
from pathlib import Path

BASE = Path.cwd()
RAW = BASE / "raw_data"
OUT = BASE / "output"
OUT.mkdir(exist_ok=True)

primary_path = RAW / "primary.xlsx"
secondary_path = RAW / "secondary.xlsx"

print("BASE:", BASE)
print("RAW exists:", RAW.exists())
print("Primary exists:", primary_path.exists())
print("Secondary exists:", secondary_path.exists())

BASE: C:\Users\dell\Desktop\Data-Analytics\School-ERP-Fee-Analysis
RAW exists: True
Primary exists: True
Secondary exists: True


In [8]:
def clean_fee_file(file_path, section_label, id_prefix):
    df = pd.read_excel(file_path)

    # Normalize headers
    df.columns = (
        df.columns.astype(str)
        .str.strip()
        .str.lower()
        .str.replace(" ", "_")
    )

    # Rename variations
    rename_map = {
        "sn": "sr",
        "sr": "sr",
        "name": "name",
        "class": "class",
        "std": "class",
        "division": "class",
        "contact": "contact",
        "fees": "fees",
        "old_balance": "old_balance",
        "oldbalance": "old_balance",
        "total": "total",
        "received": "received",
        "outstnding": "outstanding",
        "outstanding": "outstanding",
        "outstdng": "outstanding",
        "out_standing": "outstanding",
        "last_paid": "last_paid_month",
        "lastpaid": "last_paid_month",
        "last_paid_month": "last_paid_month",
    }
    df = df.rename(columns={c: rename_map.get(c, c) for c in df.columns})

    # Drop unwanted columns
    df = df.drop(columns=[c for c in ["sr", "name", "add", "action"] if c in df.columns], errors="ignore")

    # Ensure required columns
    required_cols = ["class", "contact", "fees", "old_balance", "total", "received", "last_paid_month"]
    for c in required_cols:
        if c not in df.columns:
            df[c] = np.nan if c in ["class", "contact", "last_paid_month"] else 0

    # Student ID
    df = df.reset_index(drop=True)
    df["student_id"] = [f"{id_prefix}{i+1:04d}" for i in range(len(df))]

    # Grade extraction
    df["class"] = df["class"].astype(str).str.strip()
    df["grade"] = df["class"].str.extract(r"(\d+)")[0]

    # Clean contact (first 10-digit)
    df["contact"] = df["contact"].astype(str)
    df["contact"] = df["contact"].apply(
        lambda x: re.findall(r"\d{10}", x)[0] if re.findall(r"\d{10}", x) else np.nan
    )

    # Numeric cleaning
    num_cols = ["fees", "old_balance", "total", "received"]
    for c in num_cols:
        df[c] = (
            df[c].astype(str)
            .str.replace(",", "", regex=False)
            .str.strip()
        )
        df[c] = pd.to_numeric(df[c], errors="coerce").fillna(0)

    # ✅ Recalculate outstanding (correct business logic)
    df["outstanding"] = df["total"] - df["received"]
    df.loc[df["outstanding"] < 0, "outstanding"] = 0

    # last_paid_month cleanup
    df["last_paid_month"] = df["last_paid_month"].astype(str).str.strip()
    df.loc[df["last_paid_month"].str.lower().isin(["not updated", "nan", "none", ""]), "last_paid_month"] = np.nan

    # Payment status
    df["payment_status"] = np.where(df["outstanding"] == 0, "Completed", "Pending")

    # ✅ FIXED BUCKET LABELS (ASCII only)
    def bucket(x):
        if x == 0:
            return "0"
        elif x <= 5000:
            return "1-5000"
        elif x <= 15000:
            return "5001-15000"
        else:
            return "15001+"

    df["outstanding_bucket"] = df["outstanding"].apply(bucket)

    # Contact valid
    df["contact_valid"] = np.where(
        df["contact"].astype(str).str.fullmatch(r"\d{10}"), "Yes", "No"
    )

    df["school_section"] = section_label

    return df[[
        "student_id", "grade", "class", "contact", "contact_valid",
        "fees", "old_balance", "total", "received", "outstanding",
        "payment_status", "outstanding_bucket", "last_paid_month", "school_section"
    ]]


In [10]:
primary_df = clean_fee_file(primary_path, "Primary", "S-P-")
secondary_df = clean_fee_file(secondary_path, "Secondary", "S-S-")

master_df = pd.concat([primary_df, secondary_df], ignore_index=True)

# Save outputs
master_df.to_csv(OUT / "clean_erp_fees.csv", index=False)
master_df.to_excel(OUT / "erp_fees_master.xlsx", index=False)

print("✅ CLEANING COMPLETED")
print("Total rows:", len(master_df))
master_df.head(10)


✅ CLEANING COMPLETED
Total rows: 549


,student_id,grade,class,contact,contact_valid,fees,old_balance,total,received,outstanding,payment_status,outstanding_bucket,last_paid_month,school_section
0,S-P-0001,3,3 A,7738739916,Yes,20300,0,20300,8000,12300,Pending,5001-15000,November,Primary
1,S-P-0002,1,1 A,8452864057,Yes,22000,0,22000,6500,15500,Pending,15001+,August,Primary
2,S-P-0003,1,1 A,9772328874,Yes,22000,0,22000,17700,4300,Pending,1-5000,November,Primary
3,S-P-0004,4,4 A,9833869607,Yes,22000,0,22000,8000,14000,Pending,5001-15000,July,Primary
4,S-P-0005,1,1 A,9257699914,Yes,22000,0,22000,6400,15600,Pending,15001+,June,Primary
5,S-P-0006,1,1 A,9987432037,Yes,22000,0,22000,5000,17000,Pending,15001+,June,Primary
6,S-P-0007,1,1 A,7021909550,Yes,22000,0,22000,0,22000,Pending,15001+,NaN,Primary
7,S-P-0008,1,1 A,9119229634,Yes,22000,0,22000,0,22000,Pending,15001+,NaN,Primary
8,S-P-0009,4,4 A,9769807422,Yes,22000,0,22000,11600,10400,Pending,5001-15000,November,Primary
9,S-P-0010,1,1 A,9082735507,Yes,22000,0,22000,6400,15600,Pending,15001+,July,Primary


In [11]:
print("Outstanding summary:")
print(master_df["outstanding"].describe())

print("\nPayment Status count:")
print(master_df["payment_status"].value_counts())

# Show top defaulters
master_df.sort_values("outstanding", ascending=False).head(20)


Outstanding summary:
count      549.000000
mean     15777.868852
std      10059.985562
min          0.000000
25%      10300.000000
50%      16000.000000
75%      21600.000000
max      49800.000000
Name: outstanding, dtype: float64

Payment Status count:
payment_status
Pending      471
Completed     78
Name: count, dtype: int64


,student_id,grade,class,contact,contact_valid,fees,old_balance,total,received,outstanding,payment_status,outstanding_bucket,last_paid_month,school_section
298,S-S-0123,9,9 A,9892885507,Yes,26100,23700,49800,0,49800,Pending,15001+,NaN,Secondary
424,S-S-0249,7,7 A,NaN,No,26100,23700,49800,0,49800,Pending,15001+,NaN,Secondary
222,S-S-0047,10,10 A,9322119454,Yes,26100,23700,49800,0,49800,Pending,15001+,NaN,Secondary
370,S-S-0195,8,8 A,9167782339,Yes,26100,23700,49800,0,49800,Pending,15001+,NaN,Secondary
208,S-S-0033,10,10 A,9167782339,Yes,26100,23700,49800,0,49800,Pending,15001+,NaN,Secondary
209,S-S-0034,10,10 A,8879090069,Yes,26100,20700,46800,0,46800,Pending,15001+,NaN,Secondary
451,S-S-0276,6,6 A,9833710174,Yes,26100,23700,49800,5000,44800,Pending,15001+,June,Secondary
399,S-S-0224,7,7 A,9920748172,Yes,26100,17700,43800,0,43800,Pending,15001+,NaN,Secondary
491,S-S-0316,6,6 A,9867838062,Yes,26100,17500,43600,0,43600,Pending,15001+,NaN,Secondary
228,S-S-0053,10,10 A,9892746104,Yes,90000,23700,113700,70600,43100,Pending,15001+,July,Secondary


In [6]:
print("Bucket counts:")
print(master_df["outstanding_bucket"].value_counts())

print("\nPayment status counts:")
print(master_df["payment_status"].value_counts())

# show top outstanding
master_df.sort_values("outstanding", ascending=False).head(20)


Bucket counts:
outstanding_bucket
15001+        294
5001-15000    166
0              78
1-5000         11
Name: count, dtype: int64

Payment status counts:
payment_status
Pending      471
Completed     78
Name: count, dtype: int64


,student_id,grade,class,contact,contact_valid,fees,old_balance,total,received,outstanding,payment_status,outstanding_bucket,last_paid_month,school_section
298,S-S-0123,9,9 A,9892885507,Yes,26100,23700,49800,0,49800,Pending,15001+,NaN,Secondary
424,S-S-0249,7,7 A,NaN,No,26100,23700,49800,0,49800,Pending,15001+,NaN,Secondary
222,S-S-0047,10,10 A,9322119454,Yes,26100,23700,49800,0,49800,Pending,15001+,NaN,Secondary
370,S-S-0195,8,8 A,9167782339,Yes,26100,23700,49800,0,49800,Pending,15001+,NaN,Secondary
208,S-S-0033,10,10 A,9167782339,Yes,26100,23700,49800,0,49800,Pending,15001+,NaN,Secondary
209,S-S-0034,10,10 A,8879090069,Yes,26100,20700,46800,0,46800,Pending,15001+,NaN,Secondary
451,S-S-0276,6,6 A,9833710174,Yes,26100,23700,49800,5000,44800,Pending,15001+,June,Secondary
399,S-S-0224,7,7 A,9920748172,Yes,26100,17700,43800,0,43800,Pending,15001+,NaN,Secondary
491,S-S-0316,6,6 A,9867838062,Yes,26100,17500,43600,0,43600,Pending,15001+,NaN,Secondary
228,S-S-0053,10,10 A,9892746104,Yes,90000,23700,113700,70600,43100,Pending,15001+,July,Secondary


In [ ]:
!pip install openpyxl